In [1]:
setwd("..")

In [2]:
source("Source/functions.R")
library("data.table")

In [3]:
mito_barcodes <- read.csv("Data/mito_barcodes.csv", as.is = TRUE)
mito_barcodes
mitoIDs <- mito_barcodes[, "ID"]

ID,Spacer,Barcode,Seq
<chr>,<chr>,<chr>,<chr>
M1,CGATT,ATCACG,CGATTATCACG
M2,CGTAT,CGATGT,CGTATCGATGT
M3,GCTAA,TTAGGC,GCTAATTAGGC
M4,GCTTA,TGACCA,GCTTATGACCA
M5,CGATT,ACAGTG,CGATTACAGTG
M6,CGTAT,GCCAAT,CGTATGCCAAT
M7,CGTAT,CAGATC,CGTATCAGATC
M8,CGTAT,GATCAG,CGTATGATCAG
M9,CGTAT,AGTCAA,CGTATAGTCAA


In [4]:
snv_info <- read.csv("Data/snv_loci_v2.csv", as.is = TRUE)
snv_info

SNVID,Gene,Ref,Alt,Position,Range,Start,End,Length,Distance5End,Distance3End,ForwardSeq,ReverseSeq,ForwardLen,ReverseLen
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>
SNV1,mt-Nd3,T,C,9461,9350-9534,9350,9534,185,111,73,CTGACTTTTCCTATACGTCTCCA,GGGGGAGTCAGAATGCAACTA,23,21
SNV2,mt-Nd5,C,A,12913,12783-12973,12783,12973,191,130,60,CATAGCCTGGCAGACGAACA,ATTAGTAGGGCTCAGGCGTTG,20,21
SNV3,mt-Co3,G,A,9027,8888-9093,8888,9093,206,139,66,TGCAGGATTCTTCTGAGCGT,GGGCTTGATTTATGTGGTTTCGT,20,23
SNV4,mt-Co1,G,A,6543,6409-6580,6409,6580,172,134,37,CATCCCTTGACATCGTGCTTC,AATATGATGGCGAAGTGGGCT,21,21
SNV5,mt-Tq,T,C,3816,3750-3892,3750,3892,143,66,76,AGAGGTTCAAGCCCTCTTATTT,CAACGTTTTCGGGGTATGGG,22,20
SNV6,mt-Nd6,C,T,13776,13701-13889,13701,13889,189,75,113,ACCAATCTCCCAAACCATCAAG,GGGGGATGTTGGTTGTGTTT,22,20
SNV7,mt-Nd1,G,T,3079,2970-3156,2970,3156,187,109,77,GCACCTACCCTATCACTCACAC,CGGCTCGTAAAGCTCCGAA,22,19
SNV8,D-Loop,A,G,16029,16008-16184,16008,16184,177,21,155,GTCCGCAAAACCCAATCACC,TGATCAGGACATAGGGTTTGATAGT,20,25
SNV9,mt-Co2,T,C,7612,7540-7725,7540,7725,186,72,113,AGGCCGACTAAATCAAGCAA,AGGTTAACGCTCTTAGCTTC,20,20


In [5]:
snvIDs <- snv_info[,"SNVID"]

In [6]:
chrmbases <- readLines("Data/mm10.mito/chrM.fa")[-1]
chrmbases <- paste0(chrmbases, collapse = "")
chrmbases <- strsplit(chrmbases, "")[[1]]
nchrmbases <- length(chrmbases)
str(chrmbases)

 chr [1:16299] "G" "T" "T" "A" "A" "T" "G" "T" "A" "G" "C" "T" "T" "A" "A" ...


In [7]:
MitoInfo <- fread(file = "Report/metadata/MitoInfo.csv")
head(MitoInfo)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,WellID,MitoID,HasMtDNA,HasPrimers,RCAID,MouseID,Enzyme,HasRCA,IsCtrl
<chr>,<int>,<chr>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
LB1R20P1B,733,LB1R20P1B_M7,2020-08-14,3,Neuron,1,4,Mouse02_Neuron_4,B9--M7,B9,M7,Y,Y,R20,Mouse02,None,Y,N
LB1R20P1B,733,LB1R20P1B_M9,2020-08-14,3,Neuron,1,4,Mouse02_Neuron_4,B11--M9,B11,M9,Y,Y,R20,Mouse02,None,Y,N
LB1R20P1B,733,LB1R20P1B_M10,2020-08-14,3,Neuron,1,4,Mouse02_Neuron_4,B10--M10,B10,M10,Y,Y,R20,Mouse02,None,Y,N
L3R20R23B,733,L3R20R23B_M3,2020-08-14,3,Neuron,1,6,Mouse02_Neuron_6,C1--M3/R20,C1,M3,Y,Y,R20,Mouse02,None,Y,N
L3R20R23B,733,L3R20R23B_M4,2020-08-14,3,Neuron,1,6,Mouse02_Neuron_6,C2--M4/R20,C2,M4,Y,Y,R20,Mouse02,None,Y,N
L3R20R23B,733,L3R20R23B_M5,2020-08-14,3,Neuron,1,6,Mouse02_Neuron_6,C3--M5/R20,C3,M5,Y,Y,R20,Mouse02,None,Y,N


Gather all SNV tables

In [8]:
basedifffreq_cutdemux_rerun_q30_unstranded <- read.csv("Report/SNVs/basedifffreq_cutdemux_rerun_sub500k_q30_unstranded.csv.gz", as.is = TRUE, check.names = FALSE)
dim(basedifffreq_cutdemux_rerun_q30_unstranded)

[1] 1738811      13

In [9]:
basedifffreq_cutdemux_E749_q30_unstranded <- read.csv("Report/SNVs/basedifffreq_cutdemux_E749_sub500k_q30_unstranded.csv.gz", as.is = TRUE, check.names = FALSE)
dim(basedifffreq_cutdemux_E749_q30_unstranded)

[1] 436025     13

In [10]:
basedifffreq_cutdemux_E754_q30_unstranded <- read.csv("Report/SNVs/basedifffreq_cutdemux_E754_sub500k_q30_unstranded.csv.gz", as.is = TRUE, check.names = FALSE)
dim(basedifffreq_cutdemux_E754_q30_unstranded)

[1] 352826     13

In [11]:
basedifffreq_cutdemux_E757_q30_unstranded <- read.csv("Report/SNVs/basedifffreq_cutdemux_E757_sub500k_q30_unstranded.csv.gz", as.is = TRUE, check.names = FALSE)
dim(basedifffreq_cutdemux_E757_q30_unstranded)

[1] 160858     13

In [12]:
basedifffreq_cutdemux_E771_q30_unstranded <- read.csv("Report/SNVs/basedifffreq_cutdemux_E771_sub500k_q30_unstranded.csv.gz", as.is = TRUE, check.names = FALSE)
dim(basedifffreq_cutdemux_E771_q30_unstranded)

[1] 148146     13

In [13]:
basedifffreq_cutdemux_E777_q30_unstranded <- read.csv("Report/SNVs/basedifffreq_cutdemux_E777_sub500k_q30_unstranded.csv.gz", as.is = TRUE, check.names = FALSE)
dim(basedifffreq_cutdemux_E777_q30_unstranded)

[1] 313974     13

In [14]:
basedifffreq_cutdemux_E799_q30_unstranded <- read.csv("Report/SNVs/basedifffreq_cutdemux_E799_sub500k_q30_unstranded.csv.gz", as.is = TRUE, check.names = FALSE)
dim(basedifffreq_cutdemux_E799_q30_unstranded)

[1] 67242    13

In [15]:
basedifffreq_cutdemux_q30_unstranded <- rbind(
    basedifffreq_cutdemux_rerun_q30_unstranded, 
    basedifffreq_cutdemux_E749_q30_unstranded, 
    basedifffreq_cutdemux_E754_q30_unstranded,
    basedifffreq_cutdemux_E757_q30_unstranded,
    basedifffreq_cutdemux_E771_q30_unstranded,
    basedifffreq_cutdemux_E777_q30_unstranded,
    basedifffreq_cutdemux_E799_q30_unstranded
)
setDT(basedifffreq_cutdemux_q30_unstranded)
basedifffreq_cutdemux_q30_unstranded[, LibraryMitoID := paste0(LibraryID, "_", MitoID)]
dim(basedifffreq_cutdemux_q30_unstranded)

[1] 3217882      14

In [16]:
setcolorder(basedifffreq_cutdemux_q30_unstranded, "LibraryMitoID")
setnames(basedifffreq_cutdemux_q30_unstranded, "D", "del")

Remove the old 34 mitos as well as those don't exist in the metadata

In [17]:
basedifffreq_cutdemux_q30_unstranded <- basedifffreq_cutdemux_q30_unstranded[LibraryMitoID %in% MitoInfo[, LibraryMitoID]]
dim(basedifffreq_cutdemux_q30_unstranded)

[1] 3117733      14

In [18]:
basedifffreq_cutdemux_q30_unstranded <- MitoInfo[basedifffreq_cutdemux_q30_unstranded, on = "LibraryMitoID", nomatch = NULL]
basedifffreq_cutdemux_q30_unstranded[, c("i.ExptID", "i.LibraryID", "i.MitoID") := NULL]
dim(basedifffreq_cutdemux_q30_unstranded)

[1] 3117733      29

In [19]:
head(basedifffreq_cutdemux_q30_unstranded)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,SNVID,pos,ref,depth,=,A,C,G,T,del
<chr>,<int>,<chr>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9350,C,5431,5431,0,0,0,0,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9351,T,5572,5572,0,0,0,0,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9352,G,19115,19115,0,0,0,0,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9353,A,16857,16846,0,10,0,1,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9354,C,19213,19207,1,0,0,5,0
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,SNV1,9355,T,19324,19313,1,1,0,0,9


In [ ]:
fwrite(basedifffreq_cutdemux_q30_unstranded, file = "Report/SNVs/basedifffreq_cutdemux_sub500k_q30_unstranded.csv.gz")

Gather all insertion tables

In [21]:
mpileups_ins_cutdemux_rerun <- read.csv("Report/SNVs/ins/mpileups_ins_cutdemux_rerun_sub500k.csv.gz", as.is = TRUE)
mpileups_ins_cutdemux_E749 <- read.csv("Report/SNVs/ins/mpileups_ins_cutdemux_E749_sub500k.csv.gz", as.is = TRUE)
mpileups_ins_cutdemux_E754 <- read.csv("Report/SNVs/ins/mpileups_ins_cutdemux_E754_sub500k.csv.gz", as.is = TRUE)
mpileups_ins_cutdemux_E757 <- read.csv("Report/SNVs/ins/mpileups_ins_cutdemux_E757_sub500k.csv.gz", as.is = TRUE)
mpileups_ins_cutdemux_E771 <- read.csv("Report/SNVs/ins/mpileups_ins_cutdemux_E771_sub500k.csv.gz", as.is = TRUE)
mpileups_ins_cutdemux_E777 <- read.csv("Report/SNVs/ins/mpileups_ins_cutdemux_E777_sub500k.csv.gz", as.is = TRUE)
mpileups_ins_cutdemux_E799 <- read.csv("Report/SNVs/ins/mpileups_ins_cutdemux_E799_sub500k.csv.gz", as.is = TRUE)
mpileups_ins_cutdemux <- rbind(mpileups_ins_cutdemux_rerun, 
                               mpileups_ins_cutdemux_E749, 
                               mpileups_ins_cutdemux_E754, 
                               mpileups_ins_cutdemux_E757,
                               mpileups_ins_cutdemux_E771,
                               mpileups_ins_cutdemux_E777,
                               mpileups_ins_cutdemux_E799
)
setDT(mpileups_ins_cutdemux)
dim(mpileups_ins_cutdemux)

[1] 86094    10

Remove the old 34 mitos as well as those don't exist in the metadata

In [22]:
mpileups_ins_cutdemux <- mpileups_ins_cutdemux[paste0(LibraryID, "_", MitoID) %in% MitoInfo[, LibraryMitoID]]
dim(mpileups_ins_cutdemux)

[1] 84603    10

In [23]:
mpileups_ins_cutdemux[, LibraryMitoID := paste0(LibraryID, "_", MitoID)]
mpileups_ins_cutdemux[, altperctot := altfreqtot / depth * 100]
any(setdiff(mpileups_ins_cutdemux[, LibraryMitoID], MitoInfo[, LibraryMitoID]))

[1] FALSE

In [24]:
mpileups_ins_cutdemux <- MitoInfo[mpileups_ins_cutdemux, on = "LibraryMitoID", nomatch = NULL]
mpileups_ins_cutdemux[, c("i.ExptID", "i.LibraryID", "i.MitoID") := NULL]
dim(mpileups_ins_cutdemux)

[1] 84603    27

Add per-base mt-genome stats

In [25]:
chrmproperties <- fread(file = "Report/artifact/chrmbases_properties.csv.gz")
setkey(chrmproperties, "pos")

In [26]:
mpileups_ins_cutdemux <- chrmproperties[mpileups_ins_cutdemux, on = "pos"]

In [27]:
mpileups_ins_cutdemux[, c("i.ref", "i.SNVID") := NULL]

In [28]:
setcolorder(mpileups_ins_cutdemux, colnames(mpileups_ins_cutdemux)[c(16:34, 2, 1, 3, 4:15, 35:39)])

In [29]:
colnames(mpileups_ins_cutdemux)

[1] "LibraryID"        "ExptID"           "LibraryMitoID"    "IsolationRCADate"
 [5] "DIVDays"          "CellType"         "PlateID"          "CellID"          
 [9] "CellUID"          "WellMitoBarcode"  "WellID"           "MitoID"          
[13] "HasMtDNA"         "HasPrimers"       "RCAID"            "MouseID"         
[17] "Enzyme"           "HasRCA"           "IsCtrl"           "SNVID"           
[21] "pos"              "ref"              "is_softmasked"    "nhomopolymer"    
[25] "does_new_begin"   "does_old_end"     "basefrac_bin10_A" "basefrac_bin10_C"
[29] "basefrac_bin10_G" "basefrac_bin10_T" "is_in_range"      "is_in_primer"    
[33] "dist5"            "dist3"            "depth"            "alt"             
[37] "altfreq"          "altfreqtot"       "altperctot"

In [30]:
head(mpileups_ins_cutdemux)

LibraryID,ExptID,LibraryMitoID,IsolationRCADate,DIVDays,CellType,PlateID,CellID,CellUID,WellMitoBarcode,⋯,basefrac_bin10_T,is_in_range,is_in_primer,dist5,dist3,depth,alt,altfreq,altfreqtot,altperctot
<chr>,<int>,<chr>,<IDate>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<dbl>
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,0.5454545,Y,Y,9,175,20423,CC,6,6,0.029378642
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,0.5454545,Y,Y,10,174,20424,CC,2,2,0.009792401
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,0.4545455,Y,Y,11,173,20429,"TCT,TT","1,2",3,0.014685007
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,0.4545455,Y,Y,12,172,20432,"AA,ATA","3,1",4,0.019577134
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,0.3636364,Y,Y,13,171,20448,"TA,TT","1,5",6,0.029342723
L1R24P1C,730,L1R24P1C_M1,2020-09-04,3,Neuron,1,2,Mouse05_Neuron_2,B1--M1,⋯,0.3636364,Y,Y,14,170,20451,AA,6,6,0.029338419


In [ ]:
fwrite(mpileups_ins_cutdemux, file = "Report/SNVs/ins/mpileups_ins_cutdemux_sub500k.csv.gz")

In [32]:
sessionInfo()

R version 4.4.0 (2024-04-24)
Platform: x86_64-pc-linux-gnu
Running under: openSUSE Tumbleweed

Matrix products: default
BLAS:   /home/lenovo/Applications/R/4.4.0/lib64/R/lib/libRblas.so 
LAPACK: /home/lenovo/Applications/R/4.4.0/lib64/R/lib/libRlapack.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: America/New_York
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] data.table_1.16.2

loaded via a namespace (and not attached):
 [1] digest_0.6.37     IRdisplay_1.1     utf8_1.2.4        base64enc_0.1-3  
 [5] fastmap_1.2.0     glue_1.8